# Ejercicio FASTAPI API
Para este ejercicio tendrás que desplegar un modelo de machine learning en una API para su consumo. Ya tienes un modelo entrenado,desarrollarás una API que permita consumir dicho modelo desde cualquier otra tecnología.

**Se presenta el siguiente caso de uso**

Una empresa distribuidora de ámbito nacional pretende utilizar un modelo desarrollado por el departamento de data science, con el que consiguen una predicción de las ventas a partir de los gastos en marketing de anuncios en televisión, radio y periódicos. Quieren incorporar estos datos dentro de su página web interna, donde comparten todo tipo de información relativa a resultados de la empresa, ventas, adquisiciones, etc... La web está desarrollada en AngularJS, mientras que el modelo se desarrolló en Python, por lo que precisamos de una interfaz de comunicación entre ambos sistemas.

El equipo de desarrollo necesita que implementes un microservicio para que ellos puedan consumir el modelo desde la propia web, comunicándose con una BBDD para ingestar o reentrenar el modelo. No vale base de datos en csv. El microservicio tiene que cumplir las siguientes características:
1. Ofrezca la predicción de ventas a partir de todos los valores de gastos en publicidad. (/predict)
2. Un endpoint para almacenar nuevos registros en la base de datos que deberás crear previamente.(/ingest)
3. Posibilidad de reentrenar de nuevo el modelo con los posibles nuevos registros que se recojan. (/retrain)


**NOTAS**: 
1. Deberás desplegarlo desde un repositorio de github.
2. Ojo con la ruta para hacer el load de tu modelo y datos, comprueba cual es la ruta en la que está buscándolo.
3. El desarrollo de un modelo de machine learning no es el objetivo del ejercicio, sino el desarrollo de una API con FastAPI.
4. Deberán pasar los tests para verificar que funcionan correctamente con pytest test_api.py desde terminal.

**Entregable**: repositorio github.

In [1]:
from fastapi import FastAPI, HTTPException, Depends
from pydantic import BaseModel
import sqlite3
import pickle
import os
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import uvicorn

In [14]:
# Cargar el modelo de machine learning
MODEL_PATH = "./data/advertising_model.pkl"
with open(MODEL_PATH, 'rb') as file:
    model = pickle.load(file)


# Configuración de la base de datos
df = pd.read_csv('./data/Advertising.csv')

In [3]:
import sklearn
print(sklearn.__version__)

1.2.2


In [15]:
df.drop(columns='Unnamed: 0', inplace=True)

In [16]:
df

,TV,radio,newpaper,sales
0,230.1,37.8,6s9.2,22100.0
1,44.5,39.3,45.1,10400.0
2,17.2,45.9,69.3,9300.0
3,151.5,41.3,58.5,18500.0
4,180.8,10.8,58.4,12900.0
...,...,...,...,...
195,38.2,3.7,13.8,7600.0
196,94.2,4.9,8.1,9700.0
197,177.0,9.3,6.4,12800.0
198,283.6,42.0,66.2,25500.0


In [17]:
df[df['newpaper'] == '6s9.2']

,TV,radio,newpaper,sales
0,230.1,37.8,6s9.2,22100.0


In [18]:
df['newpaper'] = df['newpaper'].str.replace('s', '').astype(float)

In [19]:
df

,TV,radio,newpaper,sales
0,230.1,37.8,69.2,22100.0
1,44.5,39.3,45.1,10400.0
2,17.2,45.9,69.3,9300.0
3,151.5,41.3,58.5,18500.0
4,180.8,10.8,58.4,12900.0
...,...,...,...,...
195,38.2,3.7,13.8,7600.0
196,94.2,4.9,8.1,9700.0
197,177.0,9.3,6.4,12800.0
198,283.6,42.0,66.2,25500.0


In [32]:
df.to_csv('./data/advertising_clean.csv')

In [37]:
df = pd.read_csv('./data/advertising_clean.csv', index_col=[0])

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   TV        200 non-null    float64
 1   radio     200 non-null    float64
 2   newpaper  200 non-null    float64
 3   sales     200 non-null    float64
dtypes: float64(4)
memory usage: 7.8 KB
